In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


import autokeras as ak

Using TensorFlow backend


In [2]:
# Load training dataset
df1 = pd.read_csv('./bioresponse/train.csv')
#df1.describe() # Visualize

# Load testing dataset
kaggle_test = pd.read_csv('./bioresponse/test.csv')
# test.describe() # Visualize

# Separate labels from training dataset for fitting
labels = df1['Activity']
del df1['Activity']

# 80/20 train test split
x_train, x_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2)
input_shape = x_train.shape
print("Shape of Input Data: " + str(input_shape))
print("Shape of Test Labels: " + str(y_test.shape))

Shape of Input Data: (3000, 1776)
Shape of Test Labels: (751,)


In [3]:
# It tries 10 different models.
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=10)
# Feed the structured data classifier with training data.
clf.fit(x_train, y_train, epochs=100)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 10 Complete [00h 17m 27s]
val_accuracy: 0.7623239159584045

Best val_accuracy So Far: 0.7940140962600708
Total elapsed time: 01h 43m 29s
Epoch 1/100
94/94 [==============================] - 30s 51ms/step - loss: 0.7021 - accuracy: 0.6320
Epoch 2/100
94/94 [==============================] - 5s 51ms/step - loss: 0.5244 - accuracy: 0.7467
Epoch 3/100
94/94 [==============================] - 5s 50ms/step - loss: 0.4627 - accuracy: 0.7870
Epoch 4/100
94/94 [==============================] - 5s 50ms/step - loss: 0.4131 - accuracy: 0.8210
Epoch 5/100
94/94 [==============================] - 5s 51ms/step - loss: 0.3762 - accuracy: 0.8367
Epoch 6/100
94/94 [==============================] - 5s 51ms/step - loss: 0.3370 - accuracy: 0.8557
Epoch 7/100
94/94 [==============================] - 7s 71ms/step - loss: 0.3235 - accuracy: 0.8703
Epoch 8/100
94/94 [==============================] - 5s 51ms/step - loss: 0.2960 - accuracy: 0.8793
Epoch 9/100
94/94 [==============================] - 5s 

INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets


24/24 [==============================] - 61s 302ms/step - loss: 3.2844 - accuracy: 0.7723
[3.2844016551971436, 0.772303581237793]


In [4]:
model = clf.export_model()
model.summary()

In [2]:
'''
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
'''

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(
    x_train, y_train, epochs=10, validation_data=(x_test, y_test)
)

NameError: name 'model' is not defined

In [ ]:
predictions = model.predict(kaggle_test)

79/79 [==============================] - 35s 401ms/step


In [ ]:
print(predictions)

[[0.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [0.]]


In [ ]:
from tensorflow.keras.models import Model

# Create a new model for inference, which includes all layers except the final classification layer
inference_model = Model(inputs=model.input, outputs=model.layers[-2].output)

# Use the inference model to get the output
probability_output = inference_model.predict(kaggle_test)

79/79 [==============================] - 434s 5s/step


In [ ]:
print(probability_output)

[[-151633.92 ]
 [ -26039.305]
 [  25842.66 ]
 ...
 [  15265.049]
 [  20277.648]
 [ -18690.914]]
